In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Actividad práctica 

### Instrucciones generales
- Escriba las rutinas de Python necesarias para resolver los problemas de cada punto
- Siga las instrucciones y conteste donde se pida
- Entregue el notebook con sus respuestas al correo: phuijse@inf.uach.cl
- Plazo de entrega: **Jueves 8 de Agosto a las 23:59**
- El trabajo es individual

### Análisis de componentes principales (PCA)

Describa en sus palabras el procedimiento de PCA sobre una matriz de datos y conteste:
1. ¿Qué optimiza PCA en términos de la proyección que se busca?
1. ¿Qué relación tiene con el problema de valores/vectores propios?
1. ¿Cómo se puede reducir la dimensión de los datos con PCA?

Considere el siguiente set de datos de imágenes de dígitos manuscritos de 8x8 píxeles
1. Reste la media de los datos y calcule la matriz de covarianza
1. Encuentre los componentes principales usando `scipy.linalg`
    1. Ordene los componentes principales de mayor a menor importancia
    1. Haga un gráfico de varianza explicada en función de la cantidad de componentes principales considerados
        1. ¿Cuántos se requieren para que se explique un 90% de la varianza?
    1. Visualize como imagen de 8x8 los 10 componentes principales de mayor importancia 
1. Haga una proyección usando los dos componentes principales de mayor importancia
    1. Grafique el resultado en un `scatter-plot` usando **distinto color** para cada digito (`label`). Debe incluir leyenda
    1. ¿Qué digitos se pueden separar en el espacio proyectado?

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
data = digits['data']
label = digits['target']
display(data.shape)

fig, ax = plt.subplots(1, 10, figsize=(8, 1), tight_layout=True)
for element, title, ax_ in zip(data[:10], label[:10], ax):
    ax_.matshow(np.reshape(element, (8, 8)), cmap=plt.cm.Greys_r)
    ax_.axis('off');
    ax_.set_title(title)

Considere las siguientes funciones para las siguientes actividades

In [ ]:
# Generar un conjunto de datos de ND datos cada uno de largo NT con distribución log-normal
def generate_data(ND, NT):
    time = np.linspace(0, 1, num=NT)
    cov = 0.1*np.exp(-0.5*(time[:, None]  - time[:, None].T)**2/0.5**2)
    data = np.exp(np.random.multivariate_normal(mean=np.zeros(NT), cov=cov, size=ND))
    return time, data

# Ajustar un modelo polinomial de cuarto grado a los datos y calcular el mse
def slow_function(time, data):
    ND, NT = data.shape
    X = np.vstack([time**k for k in range(4)]).T
    Phi = np.linalg.pinv(X)
    mse = np.zeros(shape=(ND,))
    theta = np.zeros(shape=(ND, 4))
    for i, y in enumerate(data):
        y_log = np.log(y)
        y_mean = np.mean(y_log)
        y_var = np.var(y_log)
        y_norm = (y_log - y_mean)/np.sqrt(y_var)
        theta[i, :] = np.dot(Phi, y_norm)
        model = np.dot(X, theta[i, :])
        mse[i] = np.mean((y_norm - model)**2)
    return mse, theta  

### Midiendo tiempo total

1. Para 20 valores de ND distintos generados con `np.logspace(1, 5, num=20).astype(int)`
    1. Genere un conjunto de datos de tamaño ND y largo NT=1000 usando `generate_data`
    1. Mida y guarde el tiempo total promedio (10 repeticiones) que toma ajustar un modelo polinomial a los ND datos con `slow_function`
        > HINT: Puede usar el argumento `-o` de la magia timeit para guardar el resultado
1. Use matplotlib para generar un gráfico de (tiempo total promedio) con barras de error y otro de (tiempo total promedio)/ND
    1. Estudie ambos gráficos y discuta lo que observa. ¿Qué está ocurriendo en el segundo gráfico? ¿Qué relación tiene con el overhead?

### Profiling 

Genere un conjunto de datos de tamaño 10000 y largo 1000

1. Haga un profiling con cProfile con la magia `%prun`
    1. Use los argumentos `-q -T texto` para escribir un archivo de texto con el resultado
    1. Imprima el resultado con funciones de `bash`
    1. ¿Cuáles son las 5 funciones con mayor tiempo total?
    1. ¿Cuáles son las 5 funciones con mayor tiempo acumulado?
1. Haga un profiling linea a linea con la magia `%lprun`
    1. Use el argumento `-T texto` para escribir un archivo de texto con el resultado
    1. Imprima el resultado con funciones de `bash`
    1. ¿Cuáles son las 5 lineas más costosas?


### Cythonizando

Escriba una función de Cython que retorne el mismo resultado que `slow_function`
1. Considere los resultados de su profiling para escribir versiones cythonizadas de las funciones de numpy que considere necesario
1. Importe el logaritmo y la raiz cuadrada de `math.h`
1. Use decoradores para levantar las verificaciones de Python
1. Use vistas para los arreglos de NumPy
1. Mida el tiempo total promedio (10 repeticiones) para conjuntos de datos de tamaño`N=np.logspace(1, 5, num=20).astype(int)` y largo 1000
1. Haga una gráfica de speed-up (tiempo cython/tiempo slow_function) en función de $N$
1. Estudie y discuta lo que observa

### Paralelizando

Escriba una versión paralela de su código Cython usando `parallel for` de OpenMP
1. Levante el GIL donde corresponda
1. Use un número de hilos adecuado para su computador
1. Mida el tiempo total promedio (10 repeticiones) para conjuntos de datos de tamaño`N=np.logspace(1, 5, num=20).astype(int)` y largo 1000
1. Haga una gráfica de speed-up (tiempo cython paralelo/tiempo cython secuencial) en función de $N$
1. Estudie y discuta lo que observa 